In [ ]:
!pip install kfp

In [ ]:
REGION = '###########' # change your code
ENDPOINT = '###########' # change your code
ARTIFACT_STORE_URI = '###########' # change your code

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

#  Building the docker images

In [ ]:
IMAGE_NAME = 'docker_images/transform_image'
TAG = 'latest'
TRANSFORM_IMAGE = 'gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [ ]:
!gcloud builds submit --timeout 15m --tag $TRANSFORM_IMAGE docker_images/transform_image

In [ ]:
IMAGE_NAME = 'docker_images/trainer_image'
TAG = 'latest'
TRAINER_IMAGE = 'gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [ ]:
!gcloud builds submit --timeout 15m --tag $TRAINER_IMAGE docker_images/trainer_image

In [ ]:
IMAGE_NAME = 'docker_images/base_image'
TAG = 'latest'
BASE_IMAGE = 'gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [ ]:
!gcloud builds submit --timeout 15m --tag $BASE_IMAGE docker_images/base_image

In [ ]:
IMAGE_NAME = 'docker_images/evaluate_image'
TAG = 'latest'
EVALUATE_IMAGE = 'gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [ ]:
!gcloud builds submit --timeout 15m --tag $EVALUATE_IMAGE docker_images/evaluate_image

In [ ]:
IMAGE_NAME='docker_images/kfp-cli'
TAG='latest'
KFP_CLI = 'gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [ ]:
!gcloud builds submit --timeout 15m --tag $KFP_CLI kfp-cli

# Code for custom prediction

In [ ]:
%%bash
cd custom_prediction
python setup.py sdist --formats=gztar
gsutil cp dist/my_custom_code-0.1.tar.gz gs://[your backet]/staging/dist/
cd ..

# Compiling the pipeline

In [ ]:
COMPONENT_URL_SEARCH_PREFIX = 'https://raw.githubusercontent.com/kubeflow/pipelines/1.6.0/components/gcp/'
RUNTIME_VERSION = '2.5'
PYTHON_VERSION = '3.7'

%env BASE_IMAGE={BASE_IMAGE}
%env TRANSFORM_IMAGE={TRANSFORM_IMAGE}
%env TRAINER_IMAGE={TRAINER_IMAGE}
%env EVALUATE_IMAGE = {EVALUATE_IMAGE}
%env COMPONENT_URL_SEARCH_PREFIX={COMPONENT_URL_SEARCH_PREFIX}
%env RUNTIME_VERSION={RUNTIME_VERSION}
%env PYTHON_VERSION={PYTHON_VERSION}

In [ ]:
!dsl-compile --py pipeline/model_training_pipeline.py --output pipeline/model_training_pipeline.yaml

In [ ]:
PIPELINE_NAME = 'weather_forecast_continuous_training'

!kfp --endpoint $ENDPOINT pipeline upload \
    -p $PIPELINE_NAME \
    pipeline/model_training_pipeline.yaml

In [ ]:
!kfp --endpoint $ENDPOINT pipeline list

In [ ]:
PIPELINE_ID = !(kfp --endpoint $ENDPOINT pipeline list | grep $PIPELINE_NAME | grep -o '[a-z0-9]*-[a-z0-9]*-[a-z0-9]*-[a-z0-9]*-[a-z0-9]*')
PIPELINE_ID = PIPELINE_ID[-1]

EXPERIMENT_NAME = 'Weather_Forecast_Training'
RUN_ID = 'Run_001'
SOURCE_TABLE = 'weather_data.tokyo'
NUM_EPOCHS_HYPERTUNE = '1'
NUM_EPOCHS_RETRAIN = '5'
NUM_UNITS = '256'
EVALUATION_METRIC = 'mse'
EVALUATION_METRIC_THRESHOLD = '10.0'
MODEL_ID = 'weather_forecast'
VERSION_ID = 'v01'
REPLACE_EXISTING_VERSION = 'True'

GCS_STAGING_PATH = '{}/staging'.format(ARTIFACT_STORE_URI)

In [ ]:
!kfp --endpoint $ENDPOINT run submit \
    -e $EXPERIMENT_NAME \
    -r $RUN_ID \
    -p $PIPELINE_ID \
    project_id=$PROJECT_ID \
    gcs_root=$GCS_STAGING_PATH \
    region=$REGION \
    source_table_name=$SOURCE_TABLE \
    num_epochs_hypertune=$NUM_EPOCHS_HYPERTUNE \
    num_epochs_retrain=$NUM_EPOCHS_RETRAIN \
    num_units=$NUM_UNITS \
    evaluation_metric_name=$EVALUATION_METRIC \
    evaluation_metric_threshold=$EVALUATION_METRIC_THRESHOLD \
    model_id=$MODEL_ID \
    version_id=$VERSION_ID \
    replace_existing_version=$REPLACE_EXISTING_VERSION

In [ ]:
%%bash
saved_model_cli show --dir gs://[your model_dir]/predict --all

# Manually triggering CI/CD runs

In [ ]:
SUBSTITUTIONS="""
_ENDPOINT={},\
_TRAINER_IMAGE_NAME=trainer_image,\
_TRANSFORM_IMAGE_NAME=transform_image,\
_EVALUATE_IMAGE_NAME=evaluate_image,\
_BASE_IMAGE_NAME=base_image,\
TAG_NAME=v01,\
_PIPELINE_FOLDER=.,\
_PIPELINE_DSL=model_training_pipeline.py,\
_PIPELINE_PACKAGE=model_training_pipeline.yaml,\
_PIPELINE_NAME=weather_forecast_continuous_training,\
_RUNTIME_VERSION=2.5,\
_PYTHON_VERSION=3.7,\
_COMPONENT_URL_SEARCH_PREFIX=https://raw.githubusercontent.com/kubeflow/pipelines/1.6.0/components/gcp/,\
\
_EXPERIMENT_NAME=Weather_Forecast_Training,\
_RUN_ID=Run_001,\
_GCS_STAGING_PATH=gs://[your_backet]/staging,\
_REGION=asia-northeast1,\
_SOURCE_TABLE=weather_data.tokyo,\
_NUM_EPOCHS_HYPERTUNE=1,\
_NUM_EPOCHS_RETRAIN=1,\
_NUM_UNITS=128,\
_EVALUATION_METRIC=mse,\
_EVALUATION_METRIC_THRESHOLD=10.0,\
_MODEL_ID=weather_forecast,\
_REPLACE_EXISTING_VERSION=True
""".format(ENDPOINT).strip()

In [ ]:
!gcloud builds submit . --config cloudbuild.yaml --substitutions {SUBSTITUTIONS}

# Online Prediction

In [ ]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery
from googleapiclient import errors

service = discovery.build('ml', 'v1')

def predict_json(project, model, instances, version=None):

    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [ ]:
import json

instances = {}
with open("daily_data.json", mode="r") as f:
    instances = json.load(f)
        
predict_json(PROJECT_ID, MODEL_ID, instances)